In [5]:
pip install theano

     ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.8 MB 186.2 kB/s eta 0:00:16
     ---------------------------------------- 0.0/2.8 MB 186.2 kB/s eta 0:00:16
      --------------------------------------- 0.0/2.8 MB 195.7 kB/s eta 0:00:15
     - -------------------------------------- 0.1/2.8 MB 327.3 kB/s eta 0:00:09
     - -------------------------------------- 0.1/2.8 MB 385.0 kB/s eta 0:00:08
     - -------------------------------------- 0.1/2.8 MB 359.9 kB/s eta 0:00:08
     -- ------------------------------------- 0.2/2.8 MB 426.7 kB/s eta 0:00:07
     -- ------------------------------------- 0.2/2.8 MB 452.9 kB/s eta 0:00:06
     --- ------------------------------------ 0.2/2.8 MB 537.7 kB/s eta 0:00:05
     ---- ----------------------------------- 0.3/2.8 MB 553.0 kB/s

In [9]:
pip install numpy matplotlib seaborn tensorflow-gpu


  Obtaining dependency information for seaborn from https://files.pythonhosted.org/packages/7b/e5/83fcd7e9db036c179e0352bfcd20f81d728197a16f883e7b90307a88e65e/seaborn-0.13.0-py3-none-any.whl.metadata
  Using cached seaborn-0.13.0-py3-none-any.whl.metadata (5.3 kB)
   ---------------------------------------- 0.0/294.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/294.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/294.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/294.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/294.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/294.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/294.6 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/294.6 kB 163.8 kB/s eta 0:00:02
   ---- ---------------------------------- 30.7/294.6 kB 163.8 kB/s eta 0:00:02
   ----- --------------------------------- 41.0/294.6 kB 151.3 kB/s eta 0:00

In [2]:
# Import all the things we need
%matplotlib inline
import os
os.environ["KERAS_BACKEND"] = "tensorflow"  # Use TensorFlow as the backend
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Specify which GPU to use
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import random

# Now you can continue with the rest of your code using the corrected imports and configurations.


In [3]:
# Load the dataset
#  from a certain local path
# Xd = pickle.load(open("C:/Users/Deepshikha Sharma/Downloads/RML2016.10a_dict.dat",'rb'))
import pickle

with open("/Users/Deepshikha Sharma/Desktop/VII Semester Minor Project/RML2016.10a_dict.dat", 'rb') as file:
    Xd = pickle.load(file, encoding='latin1')

print("Dataset imported")
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):
            lbl.append((mod,snr))

X = np.vstack(X)


Dataset imported


In [4]:
# For dataset RML2016.10a_dict, we should have data size 220000*2*128
print("Dataset formatted into shape: ",X.shape)
# print out the snrs and mods
print("Dataset with SNRs: ",snrs)
print("Dataset with Modulations: ",mods)
print("Data prepared")

Dataset formatted into shape:  (220000, 2, 128)
Dataset with SNRs:  [-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
Dataset with Modulations:  ['8PSK', 'AM-DSB', 'AM-SSB', 'BPSK', 'CPFSK', 'GFSK', 'PAM4', 'QAM16', 'QAM64', 'QPSK', 'WBFM']
Data prepared


In [5]:
# Partition the data into training and test sets of the form we can train/test on while keeping SNR and Mod labels handy for each
np.random.seed(2017)
n_examples = X.shape[0]
n_train = int(n_examples * 0.5)
train_idx = np.random.choice(range(0,n_examples), size=n_train, replace=False)
test_idx = list(set(range(0,n_examples))-set(train_idx))
X_train = X[train_idx]
X_test =  X[test_idx]
def to_onehot(yy):
    yy1 = np.zeros([len(yy), max(yy)+1])
    yy1[np.arange(len(yy)),yy] = 1
    return yy1

Y_train = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), train_idx)))
Y_test = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))


in_shp = list(X_train.shape[1:])
print (X_train.shape, in_shp)
classes = mods

(110000, 2, 128) [2, 128]


In [6]:
# Build VT-CNN2 Neural Net model using Keras primitives -- 
#  - Reshape [N,2,128] to [N,1,2,128] on input
#  - Pass through 2 2DConv/ReLu layers
#  - Pass through 2 Dense layers (ReLu and Softmax)
#  - Perform categorical cross entropy optimization

from keras.models import Sequential
from keras.layers import Reshape, ZeroPadding2D, Convolution2D, Dropout, Flatten, Dense, Activation

dr = 0.5  # dropout rate (%)
model = Sequential()
model.add(Reshape([1]+in_shp, input_shape=in_shp))
model.add(ZeroPadding2D(padding=((0, 0), (2, 2))))
model.add(Convolution2D(256, (1, 3), padding='valid', activation="relu", name="conv1", kernel_initializer='glorot_uniform'))
model.add(Dropout(dr))
model.add(ZeroPadding2D(padding=((0, 0), (1, 1))))  # Adjust padding here
model.add(Convolution2D(80, (1, 3), padding="valid", activation="relu", name="conv2", kernel_initializer='glorot_uniform'))  # Adjust kernel size here
model.add(Dropout(dr))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_normal', name="dense1"))
model.add(Dropout(dr))
model.add(Dense(len(classes), kernel_initializer='he_normal', name="dense2"))
model.add(Activation('softmax'))
model.add(Reshape([len(classes)]))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 1, 2, 128)         0         
                                                                 
 zero_padding2d (ZeroPaddin  (None, 1, 6, 128)         0         
 g2D)                                                            
                                                                 
 conv1 (Conv2D)              (None, 1, 4, 256)         98560     
                                                                 
 dropout (Dropout)           (None, 1, 4, 256)         0         
                                                                 
 zero_padding2d_1 (ZeroPadd  (None, 1, 6, 256)         0         
 ing2D)                                                          
                                                                 
 conv2 (Conv2D)              (None, 1, 4, 80)          6

In [7]:
# Set up some params 
nb_epoch = 100     # number of epochs to train on
batch_size = 1024  # training batch size

In [ ]:
# perform training ...
#   - call the main training loop in keras for our network+dataset
from keras.callbacks import ModelCheckpoint, EarlyStopping

filepath = 'CNN2_0.5.wts.h5'
history = model.fit(X_train,
                    Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=2,
                    validation_data=(X_test, Y_test),
                    callbacks=[
                        ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
                        EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
                    ])

# Re-load the best weights once training is finished
model.load_weights(filepath)

C:\new_folder\envs\new_env\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/100
108/108 - 52s - loss: 2.2488 - val_loss: 2.1301 - 52s/epoch - 479ms/step
Epoch 2/100


C:\new_folder\envs\new_env\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


108/108 - 48s - loss: 2.0556 - val_loss: 1.8853 - 48s/epoch - 448ms/step
Epoch 3/100
108/108 - 49s - loss: 1.8829 - val_loss: 1.8087 - 49s/epoch - 452ms/step
Epoch 4/100
108/108 - 52s - loss: 1.8280 - val_loss: 1.7634 - 52s/epoch - 481ms/step
Epoch 5/100
108/108 - 48s - loss: 1.7803 - val_loss: 1.7086 - 48s/epoch - 448ms/step
Epoch 6/100


In [ ]:
# Show simple version of performance
score = model.evaluate(X_test, Y_test, verbose=0, batch_size=batch_size)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0, batch_size=batch_size)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')


In [ ]:
# Show loss curves 
plt.figure()
plt.title('Training performance')
plt.plot(history.epoch, history.history['loss'], label='train loss+error')
plt.plot(history.epoch, history.history['val_loss'], label='val_error')
plt.legend()

In [ ]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues, labels=[]):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Plot confusion matrix
test_Y_hat = model.predict(X_test, batch_size=batch_size)
conf = np.zeros([len(classes),len(classes)])
confnorm = np.zeros([len(classes),len(classes)])
for i in range(0,X_test.shape[0]):
    j = list(Y_test[i,:]).index(1)
    k = int(np.argmax(test_Y_hat[i,:]))
    conf[j,k] = conf[j,k] + 1
for i in range(0,len(classes)):
    confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
plot_confusion_matrix(confnorm, labels=classes)

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Setting run_eagerly to True before building the model
tf.config.run_functions_eagerly(True)


# Plot confusion matrix
acc = {}
for snr in snrs:
    test_SNRs = map(lambda x: lbl[x][1], test_idx)
    test_X_i = X_test[np.where(np.array(test_SNRs) == snr)]
    test_Y_i = Y_test[np.where(np.array(test_SNRs) == snr)]

    # Set the run_eagerly parameter to True in the compile method
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'], run_eagerly=True)

    # Estimate classes
    test_Y_i_hat = model.predict(test_X_i)
    conf = np.zeros([len(classes), len(classes)])
    confnorm = np.zeros([len(classes), len(classes)])

    for i in range(0, test_X_i.shape[0]):
        j = list(test_Y_i[i, :]).index(1)
        k = int(np.argmax(test_Y_i_hat[i, :]))
        conf[j, k] = conf[j, k] + 1
    for i in range(0, len(classes)):
        confnorm[i, :] = conf[i, :] / np.sum(conf[i, :])
    plt.figure()
    plot_confusion_matrix(confnorm, labels=classes, title="ConvNet Confusion Matrix (SNR=%d)" % (snr))

    cor = np.sum(np.diag(conf))
    ncor = np.sum(conf) - cor
    print("Overall Accuracy: ", cor / (cor + ncor))
    acc[snr] = 1.0 * cor / (cor + ncor)